In [1]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

In [2]:
from pyspark.sql.functions import *

In [3]:
songs_df = spark.read.option('infer_schema','true').option('header','true').csv('/Users/mahi/Downloads/kkbox-music-recommendation-challenge/songs.csv')
songs_df.show(5)

+--------------------+-----------+---------+-------------------+--------------------+-----------+--------+
|             song_id|song_length|genre_ids|        artist_name|            composer|   lyricist|language|
+--------------------+-----------+---------+-------------------+--------------------+-----------+--------+
|CXoTN1eb7AI+DntdU...|     247640|      465|張信哲 (Jeff Chang)|                董貞|     何啟弘|     3.0|
|o0kFgae9QtnYgRkVP...|     197328|      444|          BLACKPINK|TEDDY|  FUTURE BO...|      TEDDY|    31.0|
|DwVvVurfpuz+XPuFv...|     231781|      465|       SUPER JUNIOR|                null|       null|    31.0|
|dKMBWoZyScdxSkihK...|     273554|      465|              S.H.E|              湯小康|     徐世珍|     3.0|
|W3bqWd3T+VeHFzHAU...|     140329|      726|           貴族精選|         Traditional|Traditional|    52.0|
+--------------------+-----------+---------+-------------------+--------------------+-----------+--------+
only showing top 5 rows



In [4]:
songs_df.select('genre_ids').distinct().count()

1047

In [5]:
songs_df.count()

2296833

In [6]:
songs_df.select('composer').distinct().count()

329875

In [7]:
train_df = spark.read.option('infer_schema','true').option('header','true').csv('/Users/mahi/Downloads/kkbox-music-recommendation-challenge/train.csv')
train_df.show(5)

+--------------------+--------------------+-----------------+-------------------+---------------+------+
|                msno|             song_id|source_system_tab| source_screen_name|    source_type|target|
+--------------------+--------------------+-----------------+-------------------+---------------+------+
|FGtllVqz18RPiwJj/...|BBzumQNXUHKdEBOB7...|          explore|            Explore|online-playlist|     1|
|Xumu+NIjS6QYVxDS4...|bhp/MpSNoqoxOIB+/...|       my library|Local playlist more| local-playlist|     1|
|Xumu+NIjS6QYVxDS4...|JNWfrrC7zNN7BdMps...|       my library|Local playlist more| local-playlist|     1|
|Xumu+NIjS6QYVxDS4...|2A87tzfnJTSWqD7gI...|       my library|Local playlist more| local-playlist|     1|
|FGtllVqz18RPiwJj/...|3qm6XTZ6MOCU11x8F...|          explore|            Explore|online-playlist|     1|
+--------------------+--------------------+-----------------+-------------------+---------------+------+
only showing top 5 rows



In [8]:
train_df.select('msno').distinct().count()

30755

In [9]:
train_df.count()

7377418

In [10]:
train_df.where(train_df.target==1).count()

3714656

In [11]:
train_df.where(train_df.target==0).count()

3662762

In [12]:
train_df_sample = train_df.sample(0.1,10)

In [13]:
train_df_sample.where(train_df_sample.target==1).count()/train_df_sample.count()

0.5036492390977118

In [14]:
train_df_sample.where(train_df_sample.target==0).count()/train_df_sample.count()

0.49635076090228825

In [15]:
#train_df_updated = train_df.select(concat_ws(' listened ', train_df.msno,train_df.song_id).alias('user-song-pair'))

In [16]:
sorted(train_df_sample.groupBy('msno').count().take(10))

[Row(msno='0ImlcZdGXwmJtYpwv7a3Hxwt+IhksSDz+ZYBFfaPANo=', count=31),
 Row(msno='8HdGEX5lHlImmy5Q6wMB0nWq8KxpCIlUz7Yq9t7rzdU=', count=40),
 Row(msno='MCmkCY0A7mQ9Bnam/Ln/bGhVzoyqai1eQ8TcugeH0PI=', count=11),
 Row(msno='VMn4RIfwYGqgC8Cxq39kaOLVGiR6T9+almmLGooO7Uk=', count=53),
 Row(msno='VNBKSYAB2rPCwRr9qF7k2DWahB5a7/vUOnCjdMNKQ4E=', count=8),
 Row(msno='buKhmin3ZO/WIqG2Tq27s5qzpz1rNuoXczuMA6Kj8hI=', count=23),
 Row(msno='cjrLqFlFZ5TlqIaiwzJMmde5o1nYKbNOGPA/FBR/rtU=', count=7),
 Row(msno='dLFvzIVbfES/pWPNYW4RP/NdP01kI3FGv/G2kpbO2sg=', count=37),
 Row(msno='jdJAk1ldbv+15mziwsvfc4/Hoz7kdW9b8F7OvY1Hog8=', count=38),
 Row(msno='vb2J2H2GlF6ensG79o/Gdnku4AqJ4OyBdvDzRrjgnd8=', count=30)]

In [17]:
train_df_sample.groupBy('msno','song_id','target').count().orderBy(['msno', 'target'], ascending=False).take(10)

[Row(msno='zzqc2ja7z10FtSpagYVcAZXg/gPRq7wcDZuNFj+zJSU=', song_id='ES86JRfxcI9nxEfNv70Y772KQc9gxpykvAD7XRTsBgM=', target='1', count=1),
 Row(msno='zzqc2ja7z10FtSpagYVcAZXg/gPRq7wcDZuNFj+zJSU=', song_id='4/1fO/IntKU59fWJxjhDoVBUPJ3uXTtWpUzKlzdPoQs=', target='1', count=1),
 Row(msno='zzqc2ja7z10FtSpagYVcAZXg/gPRq7wcDZuNFj+zJSU=', song_id='51ipy1Ml1KpQS7Mc7LP4QibZpxvem1o80A07gnAjM4U=', target='1', count=1),
 Row(msno='zzqc2ja7z10FtSpagYVcAZXg/gPRq7wcDZuNFj+zJSU=', song_id='quxZPlPUmS8QsJcSjHdn6zE1quBWk7w4ii3rLgOaawQ=', target='1', count=1),
 Row(msno='zzqc2ja7z10FtSpagYVcAZXg/gPRq7wcDZuNFj+zJSU=', song_id='4/f7S7iNOygpoPNxUzMv7jOPLg4HGYhpbEO7i41zUzU=', target='1', count=1),
 Row(msno='zzqc2ja7z10FtSpagYVcAZXg/gPRq7wcDZuNFj+zJSU=', song_id='CNeGIXu4+gHp5jyQDHSvonIR4modC1vHXP7b0AytDsM=', target='1', count=1),
 Row(msno='zzqc2ja7z10FtSpagYVcAZXg/gPRq7wcDZuNFj+zJSU=', song_id='L6w2d0w84FjTvFr+BhMfgu7dZAsGiOqUGmvvxIG3gvQ=', target='1', count=1),
 Row(msno='zzqc2ja7z10FtSpagYVcAZXg/gPRq7wcDZuNF

In [18]:
songs_df.show(5)

+--------------------+-----------+---------+-------------------+--------------------+-----------+--------+
|             song_id|song_length|genre_ids|        artist_name|            composer|   lyricist|language|
+--------------------+-----------+---------+-------------------+--------------------+-----------+--------+
|CXoTN1eb7AI+DntdU...|     247640|      465|張信哲 (Jeff Chang)|                董貞|     何啟弘|     3.0|
|o0kFgae9QtnYgRkVP...|     197328|      444|          BLACKPINK|TEDDY|  FUTURE BO...|      TEDDY|    31.0|
|DwVvVurfpuz+XPuFv...|     231781|      465|       SUPER JUNIOR|                null|       null|    31.0|
|dKMBWoZyScdxSkihK...|     273554|      465|              S.H.E|              湯小康|     徐世珍|     3.0|
|W3bqWd3T+VeHFzHAU...|     140329|      726|           貴族精選|         Traditional|Traditional|    52.0|
+--------------------+-----------+---------+-------------------+--------------------+-----------+--------+
only showing top 5 rows



In [19]:
train_df.show(5)

+--------------------+--------------------+-----------------+-------------------+---------------+------+
|                msno|             song_id|source_system_tab| source_screen_name|    source_type|target|
+--------------------+--------------------+-----------------+-------------------+---------------+------+
|FGtllVqz18RPiwJj/...|BBzumQNXUHKdEBOB7...|          explore|            Explore|online-playlist|     1|
|Xumu+NIjS6QYVxDS4...|bhp/MpSNoqoxOIB+/...|       my library|Local playlist more| local-playlist|     1|
|Xumu+NIjS6QYVxDS4...|JNWfrrC7zNN7BdMps...|       my library|Local playlist more| local-playlist|     1|
|Xumu+NIjS6QYVxDS4...|2A87tzfnJTSWqD7gI...|       my library|Local playlist more| local-playlist|     1|
|FGtllVqz18RPiwJj/...|3qm6XTZ6MOCU11x8F...|          explore|            Explore|online-playlist|     1|
+--------------------+--------------------+-----------------+-------------------+---------------+------+
only showing top 5 rows



In [20]:
songs_df.select('genre_ids').rdd.map(lambda x:x.genre_ids).take(15)

['465',
 '444',
 '465',
 '465',
 '726',
 '864|857|850|843',
 '458',
 '465',
 '465',
 '352|1995',
 '2157',
 '465',
 '726',
 '458',
 '359']

In [21]:
songs_df = songs_df.withColumn('genre',explode(split(songs_df.genre_ids, "\|")))
songs_df.select('genre').take(20)

[Row(genre='465'),
 Row(genre='444'),
 Row(genre='465'),
 Row(genre='465'),
 Row(genre='726'),
 Row(genre='864'),
 Row(genre='857'),
 Row(genre='850'),
 Row(genre='843'),
 Row(genre='458'),
 Row(genre='465'),
 Row(genre='465'),
 Row(genre='352'),
 Row(genre='1995'),
 Row(genre='2157'),
 Row(genre='465'),
 Row(genre='726'),
 Row(genre='458'),
 Row(genre='359'),
 Row(genre='359')]

In [22]:
train_df_cut = train_df.join(songs_df, 'song_id')
train_df_cut.show(5)

+--------------------+--------------------+-----------------+-------------------+-------------------+------+-----------+---------+------------+--------+--------+--------+-----+
|             song_id|                msno|source_system_tab| source_screen_name|        source_type|target|song_length|genre_ids| artist_name|composer|lyricist|language|genre|
+--------------------+--------------------+-----------------+-------------------+-------------------+------+-----------+---------+------------+--------+--------+--------+-----+
|++XsB7MZl/8xl/d0/...|CUtaQQ1S5S03VKAed...|       my library|Local playlist more|      local-library|     0|     452092|      947|C'est La Vie| Honey B|    null|    -1.0|  947|
|++XsB7MZl/8xl/d0/...|EbFZYurZKp8cx+mUO...|           search|        Artist more|top-hits-for-artist|     0|     452092|      947|C'est La Vie| Honey B|    null|    -1.0|  947|
|++XsB7MZl/8xl/d0/...|RoSfblbwJN/izEnFI...|         discover|   Discover Feature|song-based-playlist|     1|     45

In [23]:
train_cut_sample = train_df_cut.sample(0.01,3)

In [24]:
train_cut_sample.count()

75846

In [25]:
train_cut_sample.groupBy('msno','song_id','genre').count().take(10)

[Row(msno='RoSfblbwJN/izEnFIVw8TgOpm8R/NEpUC84Oz/b32HQ=', song_id='++XsB7MZl/8xl/d0/QFmZ4TWIRDKhAQ5saiqmjznmi4=', genre='947', count=1),
 Row(msno='iJvXG1nt/tdFPK8vKBqZMZ6e1X7c07QnyBX4d4eaTgs=', song_id='+KgAKlO7+zYsMtKzuTe3nH+iXXblehvzTNZIt4DXQ9U=', genre='2122', count=1),
 Row(msno='EfcNnWocQIAEn5gw6sHktzvZXBC9c+u2HpuzrX5leOA=', song_id='+tGKT7z6HuwhZa8mRI9hE+rQDgQFZhtVCGVrQtt7bVw=', genre='465', count=1),
 Row(msno='pXrqb5LRcKCG83LnFoa20jebHP4/gRnEvzq8Ik22jzY=', song_id='0yfLxyewedjmipOOlVzrr/Od05t5Sy+TeFQE3TWFVPA=', genre='465', count=1),
 Row(msno='flouUtxucgcV+QCOOCqv09HJCI77R36fhmTDG2ZLhwc=', song_id='1H7Za3rqrfGZhzWonEKkW3BR1LfJaWlErO8nNZaldZM=', genre='444', count=1),
 Row(msno='0D+9iiB5E2zcWdnMf6BmwoihK+oIqsnRvDog21wHnec=', song_id='1H7Za3rqrfGZhzWonEKkW3BR1LfJaWlErO8nNZaldZM=', genre='444', count=1),
 Row(msno='lc8/HRnjiaPsc/cZ6o6f6uKCAwn1opP43648Mx5dSe4=', song_id='1H7Za3rqrfGZhzWonEKkW3BR1LfJaWlErO8nNZaldZM=', genre='444', count=1),
 Row(msno='HZykoHL6T7YcO50XiCxQfrt+aQAjT

In [26]:
user_genre = train_cut_sample.select('msno','genre').groupBy('msno','genre').count().rdd.map(lambda x: x).toDF(['User', 'genre', 'genre_interest'])
user_genre.orderBy('genre_interest', ascending=False).show(5)

+--------------------+-----+--------------+
|                User|genre|genre_interest|
+--------------------+-----+--------------+
|KGXNZ/H3VxvET/+rG...|  465|            30|
|MXIMDXO0j3UpaT7Fv...|  465|            28|
|cqjRBV/jWN2ujhc+z...|  465|            25|
|JkQacE3rvmhh65R04...|  465|            24|
|mGDObQQojFOJfK2rJ...|  465|            23|
+--------------------+-----+--------------+
only showing top 5 rows



In [27]:
user_artist = train_cut_sample.select('msno','artist_name').groupBy('msno','artist_name').count().rdd.map(lambda x: x).toDF(['User', 'artist_name', 'artist_interest'])
user_artist.orderBy('artist_interest', ascending=False).show(5)

+--------------------+-------------------------+---------------+
|                User|              artist_name|artist_interest|
+--------------------+-------------------------+---------------+
|cg775T0DlPU4gthIR...|        江蕙 (Jody Jiang)|              6|
|nkSobo2abUYPOD4T2...|          五月天 (Mayday)|              6|
|2tmUzRCcD0l3et0ck...|証聲音樂圖書館 ECHO MUSIC|              6|
|MXIMDXO0j3UpaT7Fv...|証聲音樂圖書館 ECHO MUSIC|              6|
|l49SPru9KzNKnBIzi...|            吳兆南+魏龍豪|              6|
+--------------------+-------------------------+---------------+
only showing top 5 rows



In [28]:
user_language = train_cut_sample.select('msno','language').groupBy('msno','language').count().rdd.map(lambda x: x).toDF(['User', 'language', 'language_interest'])
user_language.orderBy('language_interest', ascending=False).show(5)

+--------------------+--------+-----------------+
|                User|language|language_interest|
+--------------------+--------+-----------------+
|MXIMDXO0j3UpaT7Fv...|    52.0|               51|
|JkQacE3rvmhh65R04...|     3.0|               40|
|SZ5NNypqaTWljFO1H...|    52.0|               32|
|cqjRBV/jWN2ujhc+z...|     3.0|               31|
|KGXNZ/H3VxvET/+rG...|     3.0|               30|
+--------------------+--------+-----------------+
only showing top 5 rows



In [29]:
genre_pop = train_cut_sample.select('genre','song_id','msno').groupBy('genre','song_id').count().rdd.map(lambda x: x).toDF(['genre', 'song', 'song_popularity'])
genre_pop.orderBy('song_popularity', ascending=False).show(5)

+-----+--------------------+---------------+
|genre|                song|song_popularity|
+-----+--------------------+---------------+
|  465|wBTWuHbjdjxnG1lQc...|            143|
|  458|FynUyq0+drmIARmK1...|            141|
|  458|reXuGcEWDDCnL0K3T...|            139|
|  451|YN4T/yvvXtYrBVN8K...|            135|
|  458|T86YHdD4C9JSc274b...|            121|
+-----+--------------------+---------------+
only showing top 5 rows



In [30]:
artist_pop = train_cut_sample.select('artist_name','song_id','msno').groupBy('artist_name','song_id').count().rdd.map(lambda x: x).toDF(['artist_name', 'song', 'artist_popularity'])
artist_pop.orderBy('artist_popularity', ascending=False).show(5)

+-------------+--------------------+-----------------+
|  artist_name|                song|artist_popularity|
+-------------+--------------------+-----------------+
|  Alan Walker|J4qKkLIoW7aYACuTu...|              211|
|  Alan Walker|v/3onppBGoSpGsWb8...|              186|
|  Alan Walker|zHqZ07gn+YvF36FWz...|              145|
|田馥甄 (Hebe)|wBTWuHbjdjxnG1lQc...|              143|
|  Eric 周興哲|FynUyq0+drmIARmK1...|              141|
+-------------+--------------------+-----------------+
only showing top 5 rows



In [31]:
language_pop = train_cut_sample.select('language','song_id','msno').groupBy('language','song_id').count().rdd.map(lambda x: x).toDF(['language', 'song', 'language_popularity'])
language_pop.orderBy('language_popularity', ascending=False).show(5)

+--------+--------------------+-------------------+
|language|                song|language_popularity|
+--------+--------------------+-------------------+
|    52.0|J4qKkLIoW7aYACuTu...|                211|
|    52.0|v/3onppBGoSpGsWb8...|                186|
|    52.0|zHqZ07gn+YvF36FWz...|                145|
|     3.0|wBTWuHbjdjxnG1lQc...|                143|
|     3.0|FynUyq0+drmIARmK1...|                141|
+--------+--------------------+-------------------+
only showing top 5 rows



#User versus features
features = genre, artist_name, language

# songs versus features
# same features

In [33]:
train_cut_sample.select('genre').distinct().count()

111

In [34]:
train_cut_sample.select('msno').count()

75846

In [35]:
User_side = user_genre.join(user_artist, "User", 'outer').join(user_language, 'User','outer')
User_side.show(5)

+--------------------+-----+--------------+------------------+---------------+--------+-----------------+
|                User|genre|genre_interest|       artist_name|artist_interest|language|language_interest|
+--------------------+-----+--------------+------------------+---------------+--------+-----------------+
|+XsXM6G4UNrdF3AWA...| 1259|             1|              ZIZO|              1|    31.0|                1|
|+qr25BVs1befBijPS...|  921|             1|            玖壹壹|              1|     3.0|                2|
|+qr25BVs1befBijPS...|  921|             1|            玖壹壹|              1|    52.0|                2|
|+qr25BVs1befBijPS...|  921|             1|ZAYN| Taylor Swift|              1|     3.0|                2|
|+qr25BVs1befBijPS...|  921|             1|ZAYN| Taylor Swift|              1|    52.0|                2|
+--------------------+-----+--------------+------------------+---------------+--------+-----------------+
only showing top 5 rows



In [36]:
song_side = genre_pop.join(artist_pop, 'song', 'outer').join(language_pop, "song", 'outer')
song_side.show(5)

+--------------------+-----+---------------+------------------+-----------------+--------+-------------------+
|                song|genre|song_popularity|       artist_name|artist_popularity|language|language_popularity|
+--------------------+-----+---------------+------------------+-----------------+--------+-------------------+
|++XsB7MZl/8xl/d0/...|  947|              1|      C'est La Vie|                1|    -1.0|                  1|
|+KgAKlO7+zYsMtKzu...| 2122|              1|   Astrud Gilberto|                1|    52.0|                  1|
|+tGKT7z6HuwhZa8mR...|  465|              1|周華健 (Emil Chau)|                1|     3.0|                  1|
|0yfLxyewedjmipOOl...|  465|              1|陳零九 (Nine Chen)|                1|     3.0|                  1|
|1H7Za3rqrfGZhzWon...|  444|              3|      BANGTAN BOYS|                3|    31.0|                  3|
+--------------------+-----+---------------+------------------+-----------------+--------+-------------------+
only sh

In [47]:
from pyspark.ml.feature import StringIndexer

In [49]:
indexer = StringIndexer(inputCol = "User", outputCol="UserIndex")

In [76]:
indexed = indexer.fit(user_genre).transform(user_genre).drop('User')

In [64]:
indexed.show(5)

+--------------------+-----+--------------+---------+
|                User|genre|genre_interest|UserIndex|
+--------------------+-----+--------------+---------+
|0i7inf0avDVGvwTaO...|  465|             6|    651.0|
|z+Hwg8HsbuFiry5i3...| 2122|             2|    502.0|
|2DnvslFy8Po3vODJc...|  465|             2|   3039.0|
|qlmKslzUdQ8Ta3hbc...|  465|             2|  16734.0|
|wobMEyGx2iCkFu27/...|  458|             6|    158.0|
+--------------------+-----+--------------+---------+
only showing top 5 rows



In [77]:
indexed = indexed.selectExpr('cast(genre as int) as genre_id', 'cast(UserIndex as int)', 'genre_interest')

In [78]:
indexed.printSchema()

root
 |-- genre_id: integer (nullable = true)
 |-- UserIndex: integer (nullable = true)
 |-- genre_interest: long (nullable = true)



In [79]:
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

In [80]:
training, test = indexed.randomSplit([0.8,0.2])

In [81]:
als = ALS()\
.setMaxIter(5)\
.setRegParam(0.01)\
.setUserCol("UserIndex")\
.setItemCol("genre_id")\
.setRatingCol("genre_interest")

In [82]:
print(als.explainParams())

alpha: alpha for implicit preference (default: 1.0)
checkpointInterval: set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext. (default: 10)
coldStartStrategy: strategy for dealing with unknown or new users/items at prediction time. This may be useful in cross-validation or production scenarios, for handling user/item ids the model has not seen in the training data. Supported values: 'nan', 'drop'. (default: nan)
finalStorageLevel: StorageLevel for ALS model factors. (default: MEMORY_AND_DISK)
implicitPrefs: whether to use implicit preference (default: False)
intermediateStorageLevel: StorageLevel for intermediate datasets. Cannot be 'NONE'. (default: MEMORY_AND_DISK)
itemCol: column name for item ids. Ids must be within the integer value range. (default: item, current: genre_id)
maxIter: max number of iterations (>= 

In [84]:
alsModel = als.fit(training)

In [85]:
predictions = alsModel.transform(test)

In [96]:
alsModel.recommendForAllUsers(10).selectExpr('UserIndex', 'explode(recommendations) as recommended').show(20)

+---------+-----------------+
|UserIndex|      recommended|
+---------+-----------------+
|     1580| [843, 3.3595874]|
|     1580|  [118, 2.572069]|
|     1580| [509, 2.2984807]|
|     1580|[1082, 2.1100183]|
|     1580|[1103, 2.0543993]|
|     1580|  [352, 2.052129]|
|     1580| [857, 2.0425732]|
|     1580|[2032, 1.8959343]|
|     1580| [921, 1.8205051]|
|     1580| [275, 1.8099238]|
|     4900|[1082, 4.4741592]|
|     4900|  [880, 3.775759]|
|     4900|[2032, 3.0353377]|
|     4900| [2086, 2.933754]|
|     4900| [465, 2.9290452]|
|     4900| [873, 2.6083477]|
|     4900|[2206, 2.2229495]|
|     4900| [698, 2.1499496]|
|     4900| [275, 2.0739079]|
|     4900|[1633, 1.9721645]|
+---------+-----------------+
only showing top 20 rows

